In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
reform = Reform.from_dict({
  "gov.contrib.reconciliation.medicaid_work_requirement.senate.in_effect": {
    "2027-01-01.2100-12-31": True
  },
}, country_id="us")


In [3]:
reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

In [18]:
year = 2029
state = baseline.calculate("state_code", map_to="household", period=year)
num_dependents = baseline.calculate("tax_unit_dependents", map_to="household", period=year)
married = baseline.calculate("is_married", map_to="household", period=year)
immigration_status = baseline.calculate("immigration_status", map_to="person", period=year)
employment_income = baseline.calculate("employment_income", map_to="household", period=year)
self_employment_income = baseline.calculate("self_employment_income", map_to="household", period=year)
medicaid_baseline = baseline.calculate("medicaid_enrolled", map_to="person", period=year)
household_id = baseline.calculate("household_id", map_to="household", period=year)

In [19]:
medicaid_reform = reformed.calculate("medicaid_enrolled", map_to="person", period=year)

In [23]:
medicaid_baseline.sum()/1e6

112.61243085941472

In [22]:
medicaid_reform.sum()/1e6

0.0

In [24]:
medicaid_cost_baseline = baseline.calculate("medicaid_cost", map_to="person", period=year)
medicaid_cost_reform = reformed.calculate("medicaid_cost", map_to="person", period=year)

In [25]:
medicaid_cost_baseline.sum()/1e9

823.2432315324515

In [26]:
medicaid_cost_reform.sum()/1e9

0.0

In [17]:
data = {
    "household_id": household_id,
    "State": state,
    "Married": married,
    "employment_income": employment_income,
    "Num_Dependents": num_dependents,
    "Employment_Income": employment_income,
    "Self_Employment_Income": self_employment_income,
    "medicaid_baseline": medicaid_baseline,
    "medicaid_reform": medicaid_reform,
    "immigration_status":immigration_status,
}

df_outputs = pd.DataFrame(data)
df_outputs.head()

# Filter rows where Medicaid is True in the baseline scenario
# (works if the column is already Boolean)
df_medicaid = df_outputs[df_outputs["medicaid_baseline"]]

# ── If the column holds the strings "true"/"false" instead ──
# df_medicaid = df_outputs[df_outputs["medicaid_baseline"].str.lower() == "true"]

df_medicaid.head()



,household_id,State,Married,employment_income,Num_Dependents,Employment_Income,Self_Employment_Income,medicaid_baseline,medicaid_reform,immigration_status
4,36.0,ME,0.0,41408.097656,0.0,41408.097656,0.0,True,True,CITIZEN
6,44.0,ME,0.0,78030.335938,1.0,78030.335938,0.0,True,True,CITIZEN
7,68.0,ME,0.0,0.000000,0.0,0.000000,0.0,True,True,CITIZEN
9,78.0,ME,0.0,84532.859375,0.0,84532.859375,0.0,True,False,CITIZEN
11,85.0,ME,0.0,0.000000,0.0,0.000000,0.0,True,True,CITIZEN
